In [87]:
from awpy import Demo

dem = Demo("mouz-vs-vitality-m3-inferno.dem", verbose=False)
dem.parse(player_props=['yaw', 'inventory'])
game_times = dem.parse_ticks(other_props=['game_time'])

In [97]:
print(game_times.filter(pl.col('tick') == 5))

shape: (10, 4)
┌───────────┬──────┬───────────────────┬──────────┐
│ game_time ┆ tick ┆ steamid           ┆ name     │
│ ---       ┆ ---  ┆ ---               ┆ ---      │
│ f32       ┆ i32  ┆ u64               ┆ str      │
╞═══════════╪══════╪═══════════════════╪══════════╡
│ 961.71875 ┆ 5    ┆ 76561198063336407 ┆ Spinx    │
│ 961.71875 ┆ 5    ┆ 76561197991272318 ┆ ropz     │
│ 961.71875 ┆ 5    ┆ 76561198113666193 ┆ ZywOo    │
│ 961.71875 ┆ 5    ┆ 76561198138828475 ┆ Brollan  │
│ 961.71875 ┆ 5    ┆ 76561197989744167 ┆ apEX     │
│ 961.71875 ┆ 5    ┆ 76561197973140692 ┆ mezii    │
│ 961.71875 ┆ 5    ┆ 76561198355739212 ┆ torzsi   │
│ 961.71875 ┆ 5    ┆ 76561197978835160 ┆ flameZ   │
│ 961.71875 ┆ 5    ┆ 76561198855375325 ┆ Jimpphat │
│ 961.71875 ┆ 5    ┆ 76561198193174134 ┆ xertioN  │
└───────────┴──────┴───────────────────┴──────────┘


In [85]:

import polars as pl

# round_info = dem.rounds.filter(pl.col("round_num") == 2)
# start_tick = round_info[0, "start"]
# end_tick = round_info[0, "end"]
# tick_list = list(range(start_tick, end_tick + 1, 32))

# player_ticks = dem.ticks.filter(
#     (pl.col("round_num") == 2) & (pl.col("tick").is_in(tick_list))
# ).group_by("tick", maintain_order=True).all()




def format_clock(seconds: float) -> str:
    m = int(seconds) // 60
    s = int(seconds) % 60
    return f"{m}:{s:02}"

# clock_time = round_duration - (tick_game_time - round_start_game_time)
time = 115 - (1072.578125 - 1070.71875)

print(format_clock(time))


# print(dem.rounds)

ticks = dem.parse_ticks(other_props=['game_time'])

print(ticks.filter(pl.col('tick') == 7100 ))


# for i in game_time[::64].unique():
#     print(i)

# print(player_ticks.columns)

1:53
shape: (10, 4)
┌─────────────┬──────┬───────────────────┬──────────┐
│ game_time   ┆ tick ┆ steamid           ┆ name     │
│ ---         ┆ ---  ┆ ---               ┆ ---      │
│ f32         ┆ i32  ┆ u64               ┆ str      │
╞═════════════╪══════╪═══════════════════╪══════════╡
│ 1072.578125 ┆ 7100 ┆ 76561198063336407 ┆ Spinx    │
│ 1072.578125 ┆ 7100 ┆ 76561197991272318 ┆ ropz     │
│ 1072.578125 ┆ 7100 ┆ 76561198113666193 ┆ ZywOo    │
│ 1072.578125 ┆ 7100 ┆ 76561198138828475 ┆ Brollan  │
│ 1072.578125 ┆ 7100 ┆ 76561197989744167 ┆ apEX     │
│ 1072.578125 ┆ 7100 ┆ 76561197973140692 ┆ mezii    │
│ 1072.578125 ┆ 7100 ┆ 76561198355739212 ┆ torzsi   │
│ 1072.578125 ┆ 7100 ┆ 76561197978835160 ┆ flameZ   │
│ 1072.578125 ┆ 7100 ┆ 76561198855375325 ┆ Jimpphat │
│ 1072.578125 ┆ 7100 ┆ 76561198193174134 ┆ xertioN  │
└─────────────┴──────┴───────────────────┴──────────┘


# Per tick data
{
    tick number;

    player {
        id
        name
        x
        y
        yaw
        team
        health
        alive
    }[];
    
    activeSmokes {
        id
        x
        y
    }[];

    activeMollies {
        id
        x
        y
    }[];

    inAirGreandes{
        id
        x
        y
        popped
    }[];

    gunfire {
        id
        x
        y
        weapon?
    }[];
}

In [113]:
import time
import polars as pl

SECONDARIES = {"USP-S", "Glock", "P250", "Five-SeveN", "Dual Berettas", "Tec-9", "Desert Eagle", }
PRIMARIES = {"AK-47", "M4A1-S", "AWP", "FAMAS", "Galil AR"}
GRENADES = {"Flashbang", "Smoke Grenade", "Molotov", "High Explosive Grenade", "Decoy", "Incendiary Grenade"}

def classify_inventory(items):
    if(len(items) != 0):
        knife = items[0]  # always present, always first
    else:
        knife = None
    secondary = None
    primary = None
    grenades = []

    for item in items[1:]:  # skip knife
        if item in SECONDARIES:
            secondary = item
        elif item in PRIMARIES:
            primary = item
        elif item in GRENADES:
            grenades.append(item)

    return {
        "knife": knife,
        "secondary": secondary,
        "primary": primary,
        "grenades": grenades
    }

def format_clock(seconds: float) -> str:
    m = int(seconds) // 60
    s = int(seconds) % 60
    return f"{m}:{s:02}"

def processRoundInformation(round):
    # Step 1: Get start/end ticks for round
    round_info = dem.rounds.filter(pl.col("round_num") == round)
    start_tick = round_info[0, "freeze_end"]
    end_tick = round_info[0, "end"]
    tick_list = list(range(start_tick, end_tick + 1, 32))  # You can change step to 64 if needed

    # Step 2: Get player data only for relevant ticks
    player_ticks = dem.ticks.filter(
        (pl.col("round_num") == round) & (pl.col("tick").is_in(tick_list))
    ).group_by("tick", maintain_order=True).all()

    # Step 3: Convert player info to dict
    grouped_dict = player_ticks.to_dict(as_series=False)
    grouped_dict = {
        tick: {
            "X": grouped_dict["X"][i],
            "Y": grouped_dict["Y"][i],
            "side": grouped_dict["side"][i],
            "health": grouped_dict["health"][i],
            "name": grouped_dict["name"][i],
            "yaw": grouped_dict["yaw"][i],
            "inventory": grouped_dict.get("inventory", [])[i]  # Make sure inventory exists
        }
        for i, tick in enumerate(grouped_dict["tick"])
    }

    # Step 4: Filter smokes, mollys, and grenades
    r1_smokes = dem.smokes.filter(pl.col("round_num") == round).to_pandas()
    r1_molly = dem.infernos.filter(pl.col("round_num") == round).to_pandas()
    active_grenades = dem.grenades.filter(
        (pl.col("round_num") == round) &
        (pl.col("X").is_not_null()) &
        (pl.col("tick").is_in(tick_list))
    ).to_pandas()

    grenade_by_tick = active_grenades.groupby("tick")[["thrower", "grenade_type", "X", "Y", "entity_id"]].apply(lambda x: x.to_dict("records")).to_dict()

    # Step 5: Loop through ticks and compile output
    tick_data_list = []

    for tick in tick_list:
        player_row = grouped_dict.get(tick, {
            "name": [], "X": [], "Y": [], "side": [], "health": [], "yaw": [], "inventory": []
        })

        players = []
        for i in range(len(player_row["name"])):
            inv = classify_inventory(player_row["inventory"][i])
            players.append({
                "name": player_row["name"][i],
                "X": player_row["X"][i],
                "Y": player_row["Y"][i],
                "side": player_row["side"][i],
                "health": player_row["health"][i],
                "yaw": player_row["yaw"][i],
                **inv
            })

        active_smokes = r1_smokes.query(f"{tick} >= start_tick and {tick} <= end_tick")[["X", "Y", "start_tick", "end_tick"]].to_dict("records")
        active_molly = r1_molly.query(f"{tick} >= start_tick and {tick} <= end_tick")[["X", "Y", "start_tick", "end_tick"]].to_dict("records")
        airborne_grenades = grenade_by_tick.get(tick, [])

        # clock_time = round_duration - (tick_game_time - round_start_game_time)
        tick_game_time = game_times.filter(pl.col('tick') == tick)[0]
        round_start_game_time = game_times.filter(pl.col('tick') == start_tick)[0]
        time = 115 - (tick_game_time['game_time'] - round_start_game_time['game_time'])
        time = format_clock(time[0])

        tick_data_list.append({
            "tick": tick,
            "time": time,
            "players": players,
            "activeSmokes": active_smokes,
            "activeMolly": active_molly,
            "activeGrenades": airborne_grenades,
            "shots": []
        })

    return tick_data_list

# Test run
print(processRoundInformation(1))

[{'tick': 1274, 'time': '1:55', 'players': [{'name': 'Spinx', 'X': 2472.349853515625, 'Y': 2005.969970703125, 'side': 'ct', 'health': 100, 'yaw': -157.31117248535156, 'knife': 'M9 Bayonet', 'secondary': 'USP-S', 'primary': None, 'grenades': []}, {'name': 'ropz', 'X': -1675.6199951171875, 'Y': 351.69500732421875, 'side': 't', 'health': 100, 'yaw': -107.83836364746094, 'knife': 'Karambit', 'secondary': None, 'primary': None, 'grenades': []}, {'name': 'ZywOo', 'X': -1657.22998046875, 'Y': 419.5769958496094, 'side': 't', 'health': 100, 'yaw': -35.02784729003906, 'knife': 'Karambit', 'secondary': None, 'primary': None, 'grenades': []}, {'name': 'Brollan', 'X': 2456.830078125, 'Y': 2153.159912109375, 'side': 'ct', 'health': 100, 'yaw': -119.02450561523438, 'knife': 'Karambit', 'secondary': 'USP-S', 'primary': None, 'grenades': ['Flashbang']}, {'name': 'apEX', 'X': -1520.06005859375, 'Y': 430.8909912109375, 'side': 't', 'health': 100, 'yaw': -72.62031555175781, 'knife': 'M9 Bayonet', 'seconda

In [ ]:
SECONDARIES = {"USP-S", "Glock", "P250", "Five-SeveN", "Dual Berettas", "Tec-9", "Desert Eagle", }
PRIMARIES = {"AK-47", "M4A1-S", "AWP", "FAMAS", "Galil AR"}
GRENADES = {"Flashbang", "Smoke Grenade", "Molotov", "High Explosive Grenade", "Decoy", "Incendiary Grenade"}

def classify_inventory(items):
    if(len(items) != 0):
        knife = items[0]  # always present, always first
    else:
        knife = None
    secondary = None
    primary = None
    grenades = []

    for item in items[1:]:  # skip knife
        if item in SECONDARIES:
            secondary = item
        elif item in PRIMARIES:
            primary = item
        elif item in GRENADES:
            grenades.append(item)

    return {
        "knife": knife,
        "secondary": secondary,
        "primary": primary,
        "grenades": grenades
    }

def format_clock(seconds: float) -> str:
    m = int(seconds) // 60
    s = int(seconds) % 60
    return f"{m}:{s:02}"

def processRoundInformation(dem, game_times, round):
    # Step 1: Get start/end ticks for round
    round_info = dem.rounds.filter(pl.col("round_num") == round)
    start_tick = round_info[0, "freeze_end"]
    end_tick = round_info[0, "end"]
    tick_list = list(range(start_tick, end_tick + 1, 32))  # You can change step to 64 if needed

    # Step 2: Get player data only for relevant ticks
    player_ticks = dem.ticks.filter(
        (pl.col("round_num") == round) & (pl.col("tick").is_in(tick_list))
    ).group_by("tick", maintain_order=True).all()

    # Step 3: Convert player info to dict
    grouped_dict = player_ticks.to_dict(as_series=False)
    grouped_dict = {
        tick: {
            "X": grouped_dict["X"][i],
            "Y": grouped_dict["Y"][i],
            "side": grouped_dict["side"][i],
            "health": grouped_dict["health"][i],
            "name": grouped_dict["name"][i],
            "yaw": grouped_dict["yaw"][i],
            "inventory": grouped_dict.get("inventory", [])[i]  # Make sure inventory exists
        }
        for i, tick in enumerate(grouped_dict["tick"])
    }

    # Step 4: Filter smokes, mollys, and grenades
    r1_smokes = dem.smokes.filter(pl.col("round_num") == round).to_pandas()
    r1_molly = dem.infernos.filter(pl.col("round_num") == round).to_pandas()
    active_grenades = dem.grenades.filter(
        (pl.col("round_num") == round) &
        (pl.col("X").is_not_null()) &
        (pl.col("tick").is_in(tick_list))
    ).to_pandas()

    grenade_by_tick = active_grenades.groupby("tick")[["thrower", "grenade_type", "X", "Y", "entity_id"]].apply(lambda x: x.to_dict("records")).to_dict()

    # Step 5: Loop through ticks and compile output
    tick_data_list = []

    for tick in tick_list:
        player_row = grouped_dict.get(tick, {
            "name": [], "X": [], "Y": [], "side": [], "health": [], "yaw": [], "inventory": []
        })

        players = []
        for i in range(len(player_row["name"])):
            inv = classify_inventory(player_row["inventory"][i])
            players.append({
                "name": player_row["name"][i],
                "X": player_row["X"][i],
                "Y": player_row["Y"][i],
                "side": player_row["side"][i],
                "health": player_row["health"][i],
                "yaw": player_row["yaw"][i],
                **inv
            })

        active_smokes = r1_smokes.query(f"{tick} >= start_tick and {tick} <= end_tick")[["X", "Y", "start_tick", "end_tick"]].to_dict("records")
        active_molly = r1_molly.query(f"{tick} >= start_tick and {tick} <= end_tick")[["X", "Y", "start_tick", "end_tick"]].to_dict("records")
        airborne_grenades = grenade_by_tick.get(tick, [])

        # clock_time = round_duration - (tick_game_time - round_start_game_time)
        tick_game_time = game_times.filter(pl.col('tick') == tick)[0]
        round_start_game_time = game_times.filter(pl.col('tick') == start_tick)[0]
        time = 115 - (tick_game_time['game_time'] - round_start_game_time['game_time'])
        time = format_clock(time[0])

        tick_data_list.append({
            "tick": tick,
            "time": time,
            "players": players,
            "activeSmokes": active_smokes,
            "activeMolly": active_molly,
            "activeGrenades": airborne_grenades,
            "shots": []
        })

    return tick_data_list

def parseDemo(file):
    dem = Demo(file, verbose=False)
    dem.parse(player_props=["health", "armor_value", "yaw", 'inventory'])
    game_times = dem.parse_ticks(other_props=['game_time'])
    
    return processRoundInformation(dem, game_times, 0)

In [1]:
from typing import List, Dict, Any
import polars as pl
from awpy import Demo  # or from awpy.parser import Demo depending on your version

SECONDARIES = {"USP-S", "Glock", "P250", "Five-SeveN", "Dual Berettas", "Tec-9", "Desert Eagle"}
PRIMARIES = {"AK-47", "M4A1-S", "AWP", "FAMAS", "Galil AR"}
GRENADES = {"Flashbang", "Smoke Grenade", "Molotov", "High Explosive Grenade", "Decoy", "Incendiary Grenade"}

def classify_inventory(items: List[str]) -> Dict[str, Any]:
    knife = items[0] if items else None
    secondary = None
    primary = None
    grenades = []

    for item in items[1:]:
        if item in SECONDARIES:
            secondary = item
        elif item in PRIMARIES:
            primary = item
        elif item in GRENADES:
            grenades.append(item)

    return {
        "knife": knife,
        "secondary": secondary,
        "primary": primary,
        "grenades": grenades
    }

def format_clock(seconds: float) -> str:
    m = int(seconds) // 60
    s = int(seconds) % 60
    return f"{m}:{s:02}"

class Parser:
    def parse_demo(self, file_path: bytes):
        try:
            self.dem = Demo(file_path, verbose=False)
            self.dem.parse(player_props=["health", "armor_value", "yaw", "inventory", "team_name"])
            self.game_times = self.dem.parse_ticks(other_props=["game_time"])
            return True
        except Exception as e:
            print(f"Error parsing demo: {e}")
            return False

    def write_file(self):
        
        return ""

    def active_utils(self, df, tick):
        return df[(tick >= df["start_tick"]) & (tick <= df["end_tick"])][["X", "Y", "start_tick", "end_tick"]].to_dict("records")

    def parse_demo_round(self, dem: Demo, game_times: pl.DataFrame, round_num: int = 1) -> List[Dict[str, Any]]:
        print(f"Parsing round {round_num}")
        start = time.time()

        # Get round info and ticks
        round_info = dem.rounds.filter(pl.col("round_num") == round_num)
        start_tick = round_info[0, "freeze_end"]
        end_tick = round_info[0, "end"]
        tick_list = list(range(start_tick, end_tick + 1, 32))
        print(f"Round info fetched. Duration: {time.time() - start:.3f}s")

        # Group player ticks
        t0 = time.time()
        player_ticks = dem.ticks.filter(
            (pl.col("round_num") == round_num) & (pl.col("tick").is_in(tick_list))
        ).group_by("tick", maintain_order=True).all()
        print(f"Filtered and grouped player ticks: {time.time() - t0:.3f}s")

        # Convert to grouped dictionary
        t0 = time.time()
        grouped_dict = player_ticks.to_dict(as_series=False)
        grouped_dict = {
            tick: {
                "X": grouped_dict["X"][i],
                "Y": grouped_dict["Y"][i],
                "side": grouped_dict["side"][i],
                "health": grouped_dict["health"][i],
                "name": grouped_dict["name"][i],
                "yaw": grouped_dict["yaw"][i],
                "inventory": grouped_dict.get("inventory", [])[i]
            }
            for i, tick in enumerate(grouped_dict["tick"])
        }
        print(f"Grouped dict built: {time.time() - t0:.3f}s")

        # Load smokes, molotovs, and grenades
        t0 = time.time()
        r1_smokes = dem.smokes.filter(pl.col("round_num") == round_num).to_pandas()
        r1_molly = dem.infernos.filter(pl.col("round_num") == round_num).to_pandas()
        active_grenades = dem.grenades.filter(
            (pl.col("round_num") == round_num) & pl.col("X").is_not_null() & pl.col("tick").is_in(tick_list)
        ).to_pandas()
        print(f"Loaded grenades and utilities: {time.time() - t0:.3f}s")

        # Group grenades
        t0 = time.time()
        grenade_by_tick = active_grenades.groupby("tick")[["thrower", "grenade_type", "X", "Y", "entity_id"]].apply(
            lambda x: x.to_dict("records")
        ).to_dict()
        print(f"Grenades grouped by tick: {time.time() - t0:.3f}s")

        # Build tick data
        game_time_dict = {row['tick']: row['game_time'] for row in game_times.iter_rows(named=True)}
        start_game_time = game_time_dict[start_tick]
        
        tick_data_list = []
        t0 = time.time()
        for tick in tick_list:
            player_row = grouped_dict.get(tick, {
                "name": [], "X": [], "Y": [], "side": [], "health": [], "yaw": [], "inventory": []
            })

            players = []
            for i in range(len(player_row["name"])):
                inv = classify_inventory(player_row["inventory"][i])
                players.append({
                    "name": player_row["name"][i],
                    "X": player_row["X"][i],
                    "Y": player_row["Y"][i],
                    "side": player_row["side"][i],
                    "health": player_row["health"][i],
                    "yaw": player_row["yaw"][i],
                    **inv
                })

            active_smokes = self.active_utils(r1_smokes, tick)
            active_molly = self.active_utils(r1_molly, tick)
            airborne_grenades = grenade_by_tick.get(tick, [])

            time_left = 115 - (game_time_dict[tick] - start_game_time)
            time_str = format_clock(time_left)

            tick_data_list.append({
                "tick": tick,
                "time": time_str,
                "players": players,
                "activeSmokes": active_smokes,
                "activeMolly": active_molly,
                "activeGrenades": airborne_grenades,
                "shots": []
            })
        print(f"Built tick_data_list: {time.time() - t0:.3f}s")

        print(f"Total time for round {round_num}: {time.time() - start:.3f}s\n")
        return tick_data_list

In [ ]:
parser = Parser()
parser.parse_demo("mouz-vs-vitality-m3-inferno.dem")

True

In [70]:
import polars as pl
import time

print(parser.parse_demo_round(parser.dem, parser.game_times))


Parsing round 1
Round info fetched. Duration: 0.000s
Filtered and grouped player ticks: 0.004s
Grouped dict built: 0.001s
Loaded grenades and utilities: 0.006s
Grenades grouped by tick: 0.014s
Built tick_data_list: 0.420s
Total time for round 1: 0.445s

[{'tick': 1274, 'time': '1:55', 'players': [{'name': 'Spinx', 'X': 2472.349853515625, 'Y': 2005.969970703125, 'side': 'ct', 'health': 100, 'yaw': -157.31117248535156, 'knife': 'M9 Bayonet', 'secondary': 'USP-S', 'primary': None, 'grenades': []}, {'name': 'ropz', 'X': -1675.6199951171875, 'Y': 351.69500732421875, 'side': 't', 'health': 100, 'yaw': -107.83836364746094, 'knife': 'Karambit', 'secondary': None, 'primary': None, 'grenades': []}, {'name': 'ZywOo', 'X': -1657.22998046875, 'Y': 419.5769958496094, 'side': 't', 'health': 100, 'yaw': -35.02784729003906, 'knife': 'Karambit', 'secondary': None, 'primary': None, 'grenades': []}, {'name': 'Brollan', 'X': 2456.830078125, 'Y': 2153.159912109375, 'side': 'ct', 'health': 100, 'yaw': -119.0